# Train first interation of CNN

In [ ]:
# the cnn module provides classes for training/predicting with various types of CNNs
from opensoundscape import CNN
from opensoundscape import BoxedAnnotations 

# other utilities and packages
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import random
import subprocess
from glob import glob
import sklearn

# set up plotting 
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize']=[15,5] #for large visuals
%config InlineBackend.figure_format = 'retina'

### set manual seeds for pytorch and python. useful for debugging, probably don't want to include in actual model training

In [3]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

# Load data

In [5]:
annotations_22 = pd.read_csv("rasi_2022_2s_labels.csv")
annotations_24 = pd.read_csv("rasi_2024_2s_labels.csv")
annotations_23 = pd.read_csv("data/annotated_detections_2023/rasi_annotated_detections_2023_no_uncertain.csv",index_col=0).drop(columns='index')

# make relative paths into absolue paths to the audio files
root="/Users/hannahnossan/Documents/Projects/SINE2024/ML_bootcamp/data/annotated_detections_2023"
annotations_23['file']=[f"{root}/{f}"  for f in annotations_23['file']] # practic4e list comprehension!

In [6]:
all([Path(one_file).exists() for one_file in annotations_23['file']])

True

In [7]:
annotations_22["RA_SI_MU"] = annotations_22[["RASI_C", "RASI_main"]].max(axis=1) 
annotations_24["RA_SI_MU"] = annotations_24[["RASI_C", "RASI_main"]].max(axis=1) 
annotations_23["RA_SI_MU"] = annotations_23["annotation"]


In [8]:
# concat
labels_df = pd.concat([annotations_22,annotations_23,annotations_24])[["file","start_time","end_time","RA_SI_MU"]]

In [9]:
from sklearn.model_selection import train_test_split
# split into 80% of files for training and 20% for validation
unique_files = labels_df.file.unique() # make a list of each file
train_files, val_files = train_test_split(unique_files,test_size=.2) # for each file, put it in the train or validation set
# haven't split up the clips, just the audio files

In [10]:
train_df=labels_df[labels_df['file'].isin(train_files)].set_index(['file','start_time','end_time']) # select rows for training if the file is in train_files 
val_df=labels_df[labels_df['file'].isin(val_files)].set_index(['file','start_time','end_time']) # select rows for training if the file is in val_files 

In [11]:
train_df

RA_SI_MU
file                                               start_time end_time          
/Users/hannahnossan/Documents/Projects/SINE2024... 0.0        2.0            0.0
                                                   2.0        4.0            1.0
                                                   4.0        6.0            0.0
                                                   6.0        8.0            0.0
                                                   8.0        10.0           0.0
...                                                                          ...
/Users/hannahnossan/Documents/Projects/SINE2024... 20.0       22.0           0.0
                                                   22.0       24.0           0.0
                                                   24.0       26.0           0.0
                                                   26.0       28.0           0.0
                                                   28.0       30.0           0.0

[5726 rows x 1 columns]

In [12]:
# Choose clip parameters
clip_duration = 2
clip_overlap = 0
min_label_overlap = 0.2

# Create CNN object !

In [13]:
# Create a CNN object designed to recognize 3-second samples
from opensoundscape import CNN

# Use resnet34 architecture
architecture = "resnet34"

# Can use this code to get your classes, if needed
class_list = list(train_df.columns)


model = CNN(
    architecture=architecture,
    classes=class_list,
    sample_duration=clip_duration,  # 3s, selected above
)

In [14]:
# Check to see if GPU is available on computer
print(f"model.device is: {model.device}")

model.device is: mps


In [ ]:
# check to see what percent of training data are negatives 
train_df.mean(0)

RA_SI_MU    0.278379
dtype: float64

In [17]:
import wandb

try:
    wandb.login()
    wandb_session = wandb.init(
        entity="kitzeslab",  # replace with your entity/group name
        project="RASI_training"
    )
except:  # if wandb.init fails, don't use wandb logging
    print("failed to create wandb session. wandb session will be None")
    wandb_session = None

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: hannahnossan (hannahnossan-university-of-pittsburgh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# update training_vrs for each round of training 
checkpoint_folder = Path("model_checkpts")
training_vrs = Path("model_checkpts/1_RASI_model")
checkpoint_folder.mkdir(exist_ok=True)
training_vrs.mkdir(exist_ok=True)

In [22]:
# %%capture --no-stdout --no-display
# Uncomment the line above to silence outputs from this cell

model.train(
    train_df,
    val_df,
    epochs=10,
    batch_size=64,
    log_interval=100,  # log progress every 100 batches
    num_workers=0,  # parallelized cpu tasks for preprocessing
    wandb_session=wandb_session,
    save_interval=10,  # save checkpoint every 10 epochs
    save_path=training_vrs,  # location to save checkpoints
)


Training Epoch 0


/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


  0%|          | 0/90 [00:00<?, ?it/s]

/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/opensoundscape/ml/cnn.py:213: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)
/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


Epoch: 0 [batch 0/90, 0.00%] 
	Epoch Running Average Loss: 0.564
	Most Recent Batch Loss: 0.564


...

/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/opensoundscape/ml/cnn.py:213: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)
/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())
/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/opensoundscape/ml/cnn.py:213: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=self.use_amp)
/Users/hannahnossan/miniconda


Validation.


  0%|          | 0/26 [00:00<?, ?it/s]


Best Model Appears at Epoch 3 with Validation score 0.844.


/Users/hannahnossan/miniconda3/envs/opso0.12.0/lib/python3.12/site-packages/google/protobuf/internal/well_known_types.py:178: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.FromDatetime(datetime.datetime.utcnow())


In [ ]:
# make predictions on validation set 
# evaluate low scoring positives, and high scoring negatives